In [1]:
import sub.plot as pl
import sub.functions as sb
from global_variables import gparam

gl = gparam()

In [2]:
condition = {
    # TF current
    'cur_tf':{'tf': -50.0e+3, 'turn': 16},
    
    # initial plasma profile
    'cur_ip':{'ip':100.0e+3, 'r0':0.65, 'z0':0.0, 'radius':0.3},
    
    # PF currents
    #'cur_pf':{'hcult16':0.0,'pf17t12':1.0e+3, 'pf26t18':1.5e+3,'pf4_1ab3_cc2':1.0e+3,'pf35_2':-0.5e+3, },
    'cur_pf':{'hcult16':0.0,'pf17t12':-1.0e+3, 'pf26t18':-3.0e+3,'pf4_1ab3_cc2':0.0,'pf35_2':0.0, },
    
    # number of coefficients
    'num_dpr':1, # dp/df
    'num_di2':1, # di2/df
    
    # calculate flux (r, z): result is set to 'fl_val'.
    'fl_pos':{'flc8':(0.1985, 0.450), 'f_im':(0.1985, 0.0), 'flc17':(0.1985, -0.450),
              'fls1':(1.374, 0.450), 'fls5':(1.374, 0.0), 'fls9':(1.374, -0.481)},
    
    'resolution': gl.get_dmat_coarse(),
    }

In [3]:
# calculate plasma equilibrium
cond = sb.calc_equilibrium(condition, verbose=1)

0 loss:  6574499829209.813
1 loss:  46533215525.062836
2 loss:  2912607168.845474
3 loss:  555402654.2128048
4 loss:  170143463.17122144
5 loss:  77444662.12472682
6 loss:  49681634.51382674
[1.40075776e+12 1.38074693e+12 1.36073611e+12 1.34072528e+12
 1.32071446e+12 1.30070363e+12 1.28069281e+12 1.26068198e+12
 1.24067116e+12 1.22066033e+12 1.20064951e+12 1.18063868e+12
 1.16062786e+12 1.14061703e+12 1.12060621e+12 1.10059538e+12
 1.08058456e+12 1.06057373e+12 1.04056291e+12 1.02055208e+12
 1.00054126e+12 9.80530430e+11 9.60519605e+11 9.40508780e+11
 9.20497955e+11 9.00487130e+11 8.80476305e+11 8.60465479e+11
 8.40454654e+11 8.20443829e+11 8.00433004e+11 7.80422179e+11
 7.60411354e+11 7.40400529e+11 7.20389704e+11 7.00378879e+11
 6.80368053e+11 6.60357228e+11 6.40346403e+11 6.20335578e+11
 6.00324753e+11 5.80313928e+11 5.60303103e+11 5.40292278e+11
 5.20281453e+11 5.00270628e+11 4.80259802e+11 4.60248977e+11
 4.40238152e+11 4.20227327e+11 4.00216502e+11 3.80205677e+11
 3.60194852e+11 

In [4]:
# calculated parameters
cond.keys()

dict_keys(['cur_tf', 'cur_ip', 'cur_pf', 'num_dpr', 'num_di2', 'fl_pos', 'resolution', 'vessel', 'flux_coil', 'jt', 'flux_jt', 'flux', 'ir_ax', 'iz_ax', 'r_ax', 'z_ax', 'conf_div', 'f_axis', 'f_surf', 'domain', 'error', 'jt_dp', 'jt_di2', 'param_dp', 'param_di2', 'iter', 'cal_result', 'pts', 'major_radius', 'minor_radius', 'elongation', 'triangularity', 'volume', 'cross_section', 'flux_normalized', 'fl_val', 'diff_pre', 'pressure', 'diff_pre_norm', 'pressure_norm', 'diff_i2', 'pol_current', 'diff_i2_norm', 'pol_current_norm', 'pressure_vol_average', 'beta_toroidal', 'coef_toroidal_flux', 'safety_factor', 'safety_factor_norm'])

In [5]:
cond['f_axis'], cond['f_surf']

(0.06867289851608548, 0.008057122082493626)

In [6]:
pl.plot_fundamental(cond)
#
# x = (f-f_axis)/(f_surf-f_axis) 
#
# case: ip > 0
# should be p > 0, i > i0
# 0 < f_surf < f_axis, thus when dx > 0 then df < 0
# shoud be: dp/df >0 (since dp/dx < 0), di^2/df > 0 (since di^2/dx < 0)
#
# case: ip < 0
# should be p > 0, i < i0
# f_axis < f_surf < 0, thus when dx > 0 then df > 0
# should be: dp/df < 0 (since dp/dx < 0), di^2/df < 0 (since di^2/dx < 0) 

In [7]:
cond['param_di2']

array([1.40075776e+12])

In [8]:
# total flux profile
pl.d_contour(cond['flux'])

In [9]:
# Plasma current profile
pl.d_contour(cond['jt'])

In [10]:
# calculate jt
import numpy as np
# r位置のマトリックスを作成
g = sb.dm_array(condition['resolution'].copy())
rm = [np.linspace(g.rmin, g.rmax, g.nr)]*g.nz
rm = np.array(rm)+10**(-7)
jt = 0
jt += 2.0*np.pi*rm*cond['diff_pre']['matrix'] # 圧力由来
jt += 10**(-7)/rm*cond['diff_i2']['matrix'] # pol cur由来
pl.contour(jt, g.rmin, g.zmin, g.dr, g.dz)
print('total current: ', jt.sum()*g.dr*g.dz)

total current:  99984.6033932657


In [11]:
# pressure
pl.d_contour(cond['pressure'])

In [12]:
# poloidal current
pl.d_contour(cond['pol_current'])

In [13]:
pl.d_heatmap(cond['safety_factor'])

In [14]:
# vessel 
pl.d_heatmap(cond['vessel'])

In [15]:
# domain
pl.d_heatmap(cond['domain'])